# Advent of code 2022

In [1]:
from collections import defaultdict

import re
import itertools as it

from pprint import pprint

In [2]:
from pyrsistent import pset, pvector

In [3]:
from functools import cache

## Part 1

In [4]:
test_input='''Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II'''

In [5]:
with open('data/day16.txt') as fIn:
    puzzle_input=fIn.read()

Start by parsing the input string.

In [6]:
def parse_input(str_in):
    
    volcano={}
    
    for (valve, flow, tunnels) in re.findall('Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.+)\n?',
                                             str_in):
        volcano[valve]={'flow':int(flow),
                        'tunnels':{t.strip() for t in tunnels.split(',')}}

    return volcano

parse_input(test_input)

{'AA': {'flow': 0, 'tunnels': {'BB', 'DD', 'II'}},
 'BB': {'flow': 13, 'tunnels': {'AA', 'CC'}},
 'CC': {'flow': 2, 'tunnels': {'BB', 'DD'}},
 'DD': {'flow': 20, 'tunnels': {'AA', 'CC', 'EE'}},
 'EE': {'flow': 3, 'tunnels': {'DD', 'FF'}},
 'FF': {'flow': 0, 'tunnels': {'EE', 'GG'}},
 'GG': {'flow': 0, 'tunnels': {'FF', 'HH'}},
 'HH': {'flow': 22, 'tunnels': {'GG'}},
 'II': {'flow': 0, 'tunnels': {'AA', 'JJ'}},
 'JJ': {'flow': 21, 'tunnels': {'II'}}}

And evaluate a solution. I'll have a solution as a list of operations.

This hasn't gone too well so far... let's set up a basic recursive evaluation and see what happens if we slap on the `functools.cache`r.

In [7]:
test_path_xx=pvector([('AA', 'start', 'AA', ['BB', 'CC', 'DD', 'EE', 'HH', 'JJ']),
                    ('AA', 'move', 'DD', ['BB', 'CC', 'DD', 'EE', 'HH', 'JJ']),
                    ('DD', 'open', 'DD', ['BB', 'CC', 'EE', 'HH', 'JJ']),
                    ('DD', 'move', 'CC', ['BB', 'CC', 'EE', 'HH', 'JJ']),
                    ('CC', 'move', 'BB', ['BB', 'CC', 'EE', 'HH', 'JJ']),
                    ('BB', 'open', 'BB', ['CC', 'EE', 'HH', 'JJ']),
                    ('BB', 'move', 'AA', ['CC', 'EE', 'HH', 'JJ']),
                    ('AA', 'move', 'II', ['CC', 'EE', 'HH', 'JJ']),
                    ('II', 'move', 'JJ', ['CC', 'EE', 'HH', 'JJ']),
                    ('JJ', 'open', 'JJ', ['CC', 'EE', 'HH']),
                    ('JJ', 'move', 'II', ['CC', 'EE', 'HH']),
                    ('II', 'move', 'AA', ['CC', 'EE', 'HH']),
                    ('AA', 'move', 'DD', ['CC', 'EE', 'HH']),
                    ('DD', 'move', 'EE', ['CC', 'EE', 'HH']),
                    ('EE', 'move', 'FF', ['CC', 'EE', 'HH']),
                    ('FF', 'move', 'GG', ['CC', 'EE', 'HH']),
                    ('GG', 'move', 'HH', ['CC', 'EE', 'HH']),
                    ('HH', 'open', 'HH', ['CC', 'EE']),
                    ('HH', 'move', 'GG', ['CC', 'EE']),
                    ('GG', 'move', 'FF', ['CC', 'EE']),
                    ('FF', 'move', 'EE', ['CC', 'EE']),
                    ('EE', 'open', 'EE', ['CC']),
                    ('EE', 'move', 'DD', ['CC']),
                    ('DD', 'move', 'CC', ['CC']),
                    ('CC', 'open', 'CC', [])])

In [8]:
test_path=[(a, b, c, pvector(d)) for (a, b, c, d) in test_path_xx]
test_path

[('AA', 'start', 'AA', pvector(['BB', 'CC', 'DD', 'EE', 'HH', 'JJ'])),
 ('AA', 'move', 'DD', pvector(['BB', 'CC', 'DD', 'EE', 'HH', 'JJ'])),
 ('DD', 'open', 'DD', pvector(['BB', 'CC', 'EE', 'HH', 'JJ'])),
 ('DD', 'move', 'CC', pvector(['BB', 'CC', 'EE', 'HH', 'JJ'])),
 ('CC', 'move', 'BB', pvector(['BB', 'CC', 'EE', 'HH', 'JJ'])),
 ('BB', 'open', 'BB', pvector(['CC', 'EE', 'HH', 'JJ'])),
 ('BB', 'move', 'AA', pvector(['CC', 'EE', 'HH', 'JJ'])),
 ('AA', 'move', 'II', pvector(['CC', 'EE', 'HH', 'JJ'])),
 ('II', 'move', 'JJ', pvector(['CC', 'EE', 'HH', 'JJ'])),
 ('JJ', 'open', 'JJ', pvector(['CC', 'EE', 'HH'])),
 ('JJ', 'move', 'II', pvector(['CC', 'EE', 'HH'])),
 ('II', 'move', 'AA', pvector(['CC', 'EE', 'HH'])),
 ('AA', 'move', 'DD', pvector(['CC', 'EE', 'HH'])),
 ('DD', 'move', 'EE', pvector(['CC', 'EE', 'HH'])),
 ('EE', 'move', 'FF', pvector(['CC', 'EE', 'HH'])),
 ('FF', 'move', 'GG', pvector(['CC', 'EE', 'HH'])),
 ('GG', 'move', 'HH', pvector(['CC', 'EE', 'HH'])),
 ('HH', 'open', 'HH

In [9]:
volcano=parse_input(test_input)

In [10]:
def day16_a(str_in):
    
    volcano=parse_input(str_in)

    def next_states(loc, remaining, clock):
        
        ns=[]

        if clock<1:
            return ns

        if loc in remaining:
            ns.append(((loc, remaining.remove(loc), clock-1), (clock-1)*volcano[loc]['flow']))

        ns.extend([((new_loc, remaining, clock-1), 0)
                   for new_loc in volcano[loc]['tunnels']])
        return ns
    
    
    @cache
    def best_path(loc, remaining, clock):

        nn=next_states(loc, remaining, clock)

        if nn:
            return max([(best_path(*next_state) + cost)
                        for (next_state, cost) in nn])
        else:
            return 0
    

    good_valves=pvector(sorted([valve for (valve, v) in volcano.items() if v['flow']>0]))

    
    return best_path('AA', good_valves, 30)
    

In [11]:
assert day16_a(test_input)==1651

In [12]:
day16_a(puzzle_input)

2124

## Part 2

Should probably be able to use the same technique... (famous last words?)

Previously, a state was location, remaining valves and time left. So now we should be able to just do a pair; one for me, one for the elephant.

I could be a bit more elegant with this, but frankly it's easier just to go through the four possible combinations of move/open:

OK, that was a non-starter. Or rather, a non-finisher. Ho, ho, ho!

Can I just partition the valves to be opened, and then give one partition to me and the other to the elephant? Only going to depth 26 now, so might not be too onerous.

In [13]:
def day16_b(str_in):
    
    volcano=parse_input(str_in)

    def next_states(loc, remaining, clock):
        
        ns=[]

        if clock<1:
            return ns

        if loc in remaining:
            ns.append(((loc, remaining.remove(loc), clock-1), (clock-1)*volcano[loc]['flow']))

        ns.extend([((new_loc, remaining, clock-1), 0)
                   for new_loc in volcano[loc]['tunnels']])
        return ns
    
    
    @cache
    def best_path(loc, remaining, clock):

        nn=next_states(loc, remaining, clock)

        if nn:
            return max([(best_path(*next_state) + cost)
                        for (next_state, cost) in nn])
        else:
            return 0
    

    good_valves=pset([valve for (valve, v) in volcano.items() if v['flow']>0])

    partitions=set(it.chain.from_iterable([[pset((pset(s), good_valves.difference(pset(s))))
                                            for s in it.combinations(good_valves, x)]
                                           for x in range(len(good_valves))]))
    
    return max([best_path('AA', me, 26)+best_path('AA', elephant, 26)
                for (me, elephant) in partitions])

In [14]:
assert day16_b(test_input)==1707

In [15]:
day16_b(puzzle_input)

2775